In [4]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import xgboost as xgb
lb = 'gender'
model_name = 'xgb_gender.model'

def xgb_acc_score(preds, dtrain):
    y_true = dtrain.get_label()
    y_pred = np.argmax(preds, axis=1)
    return [('acc', np.mean(y_true == y_pred))]

df_lr = pd.read_csv('./data/tfidf_stack_10W.csv')
df_dbow = pd.read_csv('./data/dbowd2v_stack_10W.csv')
data_df = pd.read_csv( './data/all_v2.csv')

y = np.array(data_df[lb])
print(data_df.head())
print(y)

df = pd.concat([df_lr, df_dbow], axis=1)
print(df.head())

TR = 80000
esr = 100
evals = 1
num_class = len(np.unique(y))

X ,X_te= df.iloc[:TR], df.iloc[TR:]
y ,y_te= y[:TR],y[TR:]

ss = 0.5
mc = 0.8
md = 7
gm = 1
n_trees = 25

params = {
    "objective": "multi:softprob",
    "booster": "gbtree",
    "num_class": num_class,
    'max_depth': md,
    'min_child_weight': mc,
    'subsample': ss,
    'colsample_bytree': 1,
    'gamma': gm,
    "eta": 0.01,
    "lambda": 0,
    'alpha': 0,
    "silent": 1,
}

dtrain = xgb.DMatrix(X, y)
dvalid = xgb.DMatrix(X_te, y_te)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
bst = xgb.train(params, dtrain, n_trees, evals=watchlist, feval=xgb_acc_score, maximize=True,
                early_stopping_rounds=esr, verbose_eval=evals)

bst.save_model(model_name)

                                 Id  age  gender  Education  \
0  22DD920316420BE2DF8D6EE651BA174B    1       1          4   
1  43CC3AF5A8D6430A3B572337A889AFE4    2       1          3   
2  E97654BFF5570E2CCD433EA6128EAC19    4       1          0   
3  6931EFC26D229CCFCEA125D3F3C21E57    4       2          3   
4  E780470C3BB0D340334BD08CDCC3C71A    2       2          4   

                                               query  
0  柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...  
1  广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...  
2  钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...  
3  最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...  
4  干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...  
[1 1 1 ... 2 2 2]
   tfidf_Education_0  tfidf_Education_1  tfidf_Education_2  tfidf_Education_3  \
0           0.158017           0.004670           0.005062           0.198774   
1           0.131297           0.004623           0.003296           0.205626   
2           0.104105          